This model uses Lasso regression with data from the previous three seasons for each player to generate a set of features and coefficients. These are then used to run a regression and create a projection. At this moment first, second, and third year player seasons are not included.

# Import Packages and Data

In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
# Load the data from a CSV file into a DataFrame
df = pd.read_csv('../Resources/properly_formatted_data.csv')

In [3]:
# Combine lists of plate appearance stats and non-plate appearance stats
non_pa_list = ['G', 'AB', 'PA', 'AVG', 'BB%', 'OBP', 'SLG', 'OPS', 'ISO', 'wOBA', 'wRAA', 'wRC+', 'BB%+']
pa_list = ['H', '1B', '2B', '3B', 'HR', 'R', 'RBI', 'BB', 'SH', 'SB', 'L-WAR', 'wRC', 'WAR']
combined_list = pa_list + non_pa_list

# Automating

In [4]:
# Drop all first, second, and third year seasons
df_dropped = df.dropna()

In [5]:
# Create a training model without lables
model5_df = df_dropped[['IDfg', 'Season', 'Name', 'Team', 'Age']] 

In [6]:
# Iterate through each statistic in the 'combined_list' for projection.
for stat in combined_list:
    # Isolate stat to measure
    y = df_dropped[stat]

    # Get previous three years of data to be used to find best features and coeficients
    data = df_dropped.loc[:, df_dropped.columns.str.contains('Prev')]
    x = data

    # Create and training set and a test set
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

    # Scale all data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Create lasso regression model and fit the data
    lasso = Lasso(alpha=0.1)
    lasso.fit(X_train_scaled, y_train)
    
    # Print features and coeficients
    feature_coefficients = pd.DataFrame({
        'Feature': x.columns,
        'Coefficient': lasso.coef_
    })
    print(feature_coefficients[feature_coefficients['Coefficient'] != 0])

    # Add projections to dataframe
    X_scaled = scaler.transform(data)
    projections = lasso.predict(X_scaled)
    model5_df[stat] = y
    model5_df[f'Projected_{stat}'] = projections
    

C:\Users\thoma\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.622e+04, tolerance: 1.313e+04
  model = cd_fast.enet_coordinate_descent(
C:\Users\thoma\AppData\Local\Temp\ipykernel_9008\328787670.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model5_df[stat] = y
C:\Users\thoma\AppData\Local\Temp\ipykernel_9008\328787670.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

                  Feature  Coefficient
0                 1Prev_G    -7.779536
1                 2Prev_G    -0.317883
2                 3Prev_G    -2.209901
3                1Prev_AB     7.956306
9                 1Prev_H    31.679001
10                2Prev_H     9.048889
11                3Prev_H     0.876612
14               3Prev_1B     1.500025
16               2Prev_2B     0.420717
17               3Prev_2B     2.066121
18               1Prev_3B     0.646404
19               2Prev_3B     0.377622
27              1Prev_RBI     0.972506
31               2Prev_BB    -0.821570
32               3Prev_BB    -1.066261
34               2Prev_SH    -0.718570
35               3Prev_SH    -0.887002
36               1Prev_SB     1.028720
37               2Prev_SB     0.449858
41              3Prev_AVG     0.057702
42              1Prev_BB%     0.603669
44              3Prev_BB%    -0.070121
49              2Prev_SLG     0.038682
54              1Prev_ISO     1.265835
55              2Prev_ISO

C:\Users\thoma\AppData\Local\Temp\ipykernel_9008\328787670.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model5_df[stat] = y
C:\Users\thoma\AppData\Local\Temp\ipykernel_9008\328787670.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model5_df[f'Projected_{stat}'] = projections


                 Feature  Coefficient
2                3Prev_G    -0.226259
9                1Prev_H     3.010440
15              1Prev_2B     2.393303
16              2Prev_2B     1.616111
17              3Prev_2B     0.910894
18              1Prev_3B     0.037483
19              2Prev_3B     0.054540
21              1Prev_HR     0.045410
27             1Prev_RBI     0.709658
33              1Prev_SH    -0.009099
34              2Prev_SH    -0.060059
35              3Prev_SH    -0.149098
49             2Prev_SLG     0.042057
50             3Prev_SLG     0.005120
54             1Prev_ISO     0.205567
55             2Prev_ISO     0.091639
72             1Prev_Pos    -0.040894
75             1Prev_RAR     0.596103
76             2Prev_RAR     0.082238
81             1Prev_Spd     0.429162
82             2Prev_Spd     0.205475
94             2Prev_wSB    -0.029223
100             2Prev_Lg     0.002110
109   Prev_League_Avg_AB    -0.041690
114   Prev_League_Avg_3B    -0.069245
124  Prev_Le

C:\Users\thoma\AppData\Local\Temp\ipykernel_9008\328787670.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model5_df[stat] = y
C:\Users\thoma\AppData\Local\Temp\ipykernel_9008\328787670.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model5_df[f'Projected_{stat}'] = projections


...

C:\Users\thoma\AppData\Local\Temp\ipykernel_9008\328787670.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model5_df[stat] = y
C:\Users\thoma\AppData\Local\Temp\ipykernel_9008\328787670.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model5_df[f'Projected_{stat}'] = projections


                  Feature  Coefficient
0                 1Prev_G    -3.738982
1                 2Prev_G    -3.071142
2                 3Prev_G    -4.179283
3                1Prev_AB    12.910469
4                2Prev_AB     7.090953
..                    ...          ...
126   Prev_League_Avg_ISO    -2.611922
128  Prev_League_Avg_wRAA    -0.216196
135   Prev_League_Avg_Spd     0.293034
136  Prev_League_Avg_wRC+     0.281326
140   Prev_League_Avg_Off     0.377439

[70 rows x 2 columns]


C:\Users\thoma\AppData\Local\Temp\ipykernel_9008\328787670.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model5_df[stat] = y
C:\Users\thoma\AppData\Local\Temp\ipykernel_9008\328787670.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model5_df[f'Projected_{stat}'] = projections


                  Feature  Coefficient
0                 1Prev_G     0.593203
2                 3Prev_G    -2.033216
9                 1Prev_H    -8.105605
10                2Prev_H    -0.396380
12               1Prev_1B    -0.027053
15               1Prev_2B    -0.180640
16               2Prev_2B    -0.568500
19               2Prev_3B    -1.357267
20               3Prev_3B    -0.066843
22               2Prev_HR    -2.094162
23               3Prev_HR    -1.899175
24                1Prev_R    -0.293981
25                2Prev_R    -2.760375
26                3Prev_R    -1.076078
29              3Prev_RBI    -1.841361
30               1Prev_BB    14.349472
31               2Prev_BB     8.169120
32               3Prev_BB     8.114323
33               1Prev_SH    -0.494393
34               2Prev_SH    -1.022155
36               1Prev_SB    -0.215656
37               2Prev_SB    -0.578553
38               3Prev_SB    -0.657832
39              1Prev_AVG     0.841261
42              1Prev_BB%

C:\Users\thoma\AppData\Local\Temp\ipykernel_9008\328787670.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model5_df[stat] = y
C:\Users\thoma\AppData\Local\Temp\ipykernel_9008\328787670.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model5_df[f'Projected_{stat}'] = projections


# Exporting Data

In [7]:
model5_df.head()

NameError: name 'model4_df' is not defined

In [ ]:
model5_df.to_csv('../Projection_Results/model_five.csv', index=False)